In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
df = pd.read_csv('../input/text-classification/Amazon_text.csv')
df.head()

,Review,Label
0,Stuning even for the non-gamer: This sound tr...,__label__2
1,The best soundtrack ever to anything.: I'm re...,__label__2
2,Amazing!: This soundtrack is my favorite musi...,__label__2
3,Excellent Soundtrack: I truly like this sound...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2


### Converting to numeric labels

lets convert the labels to 0 and 1

In [3]:
def sent(x):
    if(x == '__label__1'):
        return 0
    else :
        return 1

In [4]:
df['Sentiment'] = df['Label'].apply(sent)

In [5]:
df.head(10)

,Review,Label,Sentiment
0,Stuning even for the non-gamer: This sound tr...,__label__2,1
1,The best soundtrack ever to anything.: I'm re...,__label__2,1
2,Amazing!: This soundtrack is my favorite musi...,__label__2,1
3,Excellent Soundtrack: I truly like this sound...,__label__2,1
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2,1
5,an absolute masterpiece: I am quite sure any ...,__label__2,1
6,"Buyer beware: This is a self-published book, ...",__label__1,1
7,Glorious story: I loved Whisper of the wicked...,__label__2,1
8,A FIVE STAR BOOK: I just finished reading Whi...,__label__2,1
9,Whispers of the Wicked Saints: This was a eas...,__label__2,1


In [6]:
len('__label__2')

10

In [7]:
len((df.loc[0].Label))

11

The reason I was not able to apply 'apply' was because the dataset itself is labelled with ne extra space.

In [8]:
df['Sentiment']= np.where(df['Label'] == '__label__1 ', 0, 1)

In [9]:
df.head(10)

,Review,Label,Sentiment
0,Stuning even for the non-gamer: This sound tr...,__label__2,1
1,The best soundtrack ever to anything.: I'm re...,__label__2,1
2,Amazing!: This soundtrack is my favorite musi...,__label__2,1
3,Excellent Soundtrack: I truly like this sound...,__label__2,1
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2,1
5,an absolute masterpiece: I am quite sure any ...,__label__2,1
6,"Buyer beware: This is a self-published book, ...",__label__1,0
7,Glorious story: I loved Whisper of the wicked...,__label__2,1
8,A FIVE STAR BOOK: I just finished reading Whi...,__label__2,1
9,Whispers of the Wicked Saints: This was a eas...,__label__2,1


### Text preprocessing

We use a regular expression to remove unwanted symbols , characters and numbers.

In [11]:
import re
def clean_reviews(text):
    text=re.sub("[^a-zA-Z]"," ",str(text))
    return re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)

In [12]:
df['Summary'] = df.Review.apply(clean_reviews)
df.head(10)

,Review,Label,Sentiment,Summary
0,Stuning even for the non-gamer: This sound tr...,__label__2,1,Stuning even for the non gamer This sound tr...
1,The best soundtrack ever to anything.: I'm re...,__label__2,1,The best soundtrack ever to anything I m re...
2,Amazing!: This soundtrack is my favorite musi...,__label__2,1,Amazing This soundtrack is my favorite musi...
3,Excellent Soundtrack: I truly like this sound...,__label__2,1,Excellent Soundtrack I truly like this sound...
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2,1,Remember Pull Your Jaw Off The Floor After H...
5,an absolute masterpiece: I am quite sure any ...,__label__2,1,an absolute masterpiece I am quite sure any ...
6,"Buyer beware: This is a self-published book, ...",__label__1,0,Buyer beware This is a self published book ...
7,Glorious story: I loved Whisper of the wicked...,__label__2,1,Glorious story I loved Whisper of the wicked...
8,A FIVE STAR BOOK: I just finished reading Whi...,__label__2,1,A FIVE STAR BOOK I just finished reading Whi...
9,Whispers of the Wicked Saints: This was a eas...,__label__2,1,Whispers of the Wicked Saints This was a eas...


In [13]:
len(df.Summary[8])

525

### Splitting into input and output data

before tokenization


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
X = df.Summary
y = df.Sentiment

tokenizer = Tokenizer(num_words=10000, oov_token='xxxxxxx')

In [16]:
tokenizer.fit_on_texts(X)
X_dict = tokenizer.word_index
print(len(X_dict))
#X_dict.items()

3288


Applying tokenizer to entire sequences

In [17]:
X_seq = tokenizer.texts_to_sequences(X)

Lets use padding to make vectors of equal length 500

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
X_padded_seq = pad_sequences(X_seq, padding='post', maxlen=500)

In [20]:
X_padded_seq.shape

(199, 500)

Lets convert target variable in Panda series to a numpy array

In [21]:
type(y)

pandas.core.series.Series

In [22]:
y = np.array(y)
y = y.flatten()
y.shape

(199,)

In [23]:
type(y)

numpy.ndarray

## Creating a Deep Learning Model

In [24]:
import tensorflow as tf

In [29]:
num_epochs = 10
text_model = tf.keras.Sequential([tf.keras.layers.Embedding(input_length=500, input_dim=10000, output_dim=50),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(6, activation='relu'),
                                 tf.keras.layers.Dense(1, activation='sigmoid')])

In [30]:
text_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
text_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           500000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 150006    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 650,013
Trainable params: 650,013
Non-trainable params: 0
_________________________________________________________________


In [31]:
text_model.fit(X_padded_seq,y, epochs=num_epochs)

Epoch 1/10
7/7 [==============================] - 1s 11ms/step - loss: 0.6941 - accuracy: 0.5137
Epoch 2/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6910 - accuracy: 0.4749
Epoch 3/10
7/7 [==============================] - 0s 9ms/step - loss: 0.6932 - accuracy: 0.4509
Epoch 4/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6929 - accuracy: 0.5932
Epoch 5/10
7/7 [==============================] - 0s 7ms/step - loss: 0.6926 - accuracy: 0.5902
Epoch 6/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6929 - accuracy: 0.5282
Epoch 7/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6929 - accuracy: 0.5241
Epoch 8/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6926 - accuracy: 0.5425
Epoch 9/10
7/7 [==============================] - 0s 7ms/step - loss: 0.6927 - accuracy: 0.5259
Epoch 10/10
7/7 [==============================] - 0s 8ms/step - loss: 0.6915 - accuracy: 0.5853


In [32]:
embeddings = text_model.layers[0]
embeddings.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 50) dtype=float32, numpy=
 array([[ 0.04079622,  0.02292706, -0.03232499, ..., -0.03182842,
         -0.03683313,  0.00537353],
        [ 0.03356124,  0.03305756,  0.01425945, ..., -0.02824062,
         -0.00416134,  0.00383566],
        [-0.017254  , -0.04495968, -0.03020482, ...,  0.04481029,
          0.03026899, -0.00586095],
        ...,
        [-0.03299543,  0.00093194,  0.01611394, ...,  0.04606671,
          0.03654328,  0.04285501],
        [ 0.02474617, -0.00797722,  0.00833732, ..., -0.02570814,
          0.01523323,  0.02978856],
        [-0.01404494,  0.01318273,  0.00843749, ...,  0.03220427,
         -0.0491592 , -0.03932636]], dtype=float32)>]

In [33]:
weights = embeddings.get_weights()[0]
print(weights.shape)

(10000, 50)


inorder to visualizethe embeddings in 2D space, we must reverse the key value for embeddings and respective words so as to represent every word via its embedding. to do this, we create a helper function.

In [34]:
index_based_embedding = dict([(value, key) for (key, value) in X_dict.items()])

In [35]:
def decode_review(text):
    return ' ',join([index_based_embedding.get(i, '?') for i in text])

In [36]:
index_based_embedding[1]

'xxxxxxx'

In [37]:
index_based_embedding[2]

'the'

In [38]:
weights[1]

array([ 0.03356124,  0.03305756,  0.01425945,  0.01185899, -0.01546506,
       -0.02608928, -0.01662165,  0.00568872,  0.049253  , -0.03674727,
       -0.03021718, -0.03310522,  0.03091725,  0.01169325,  0.02491578,
       -0.03937102, -0.01564944, -0.00131901, -0.04968838, -0.02535038,
        0.03204725,  0.04338208,  0.00362326, -0.03988997,  0.01023174,
       -0.04725772,  0.0407236 , -0.04607836,  0.00545069, -0.01283407,
       -0.03573358,  0.01117269, -0.00716975, -0.02515155, -0.02426187,
       -0.04167993,  0.03244768,  0.00733743, -0.03203829, -0.04990609,
       -0.0242051 , -0.0253855 , -0.0437688 ,  0.00729066, -0.03006513,
       -0.00896826,  0.03378517, -0.02824062, -0.00416134,  0.00383566],
      dtype=float32)

In the final part of this exercise, we extract the embeddings value and 
put it into a .tsv file, along with another .tsv file that captures the words 
of the embedding.

In [42]:
import io

In [44]:
vec = io.open('embedding_vectors_new.tsv', 'w', encoding='utf-8')
meta = io.open('metadata_new.tsv', 'w', encoding='utf-8')

for i in range(1, len(X_dict)): # x_dict == vocabsize
    word = index_based_embedding[i]
    embedding_vec_values = weights[i]
    meta.write(word + "\n")
    vec.write('\t'.join([str(x) for x in embedding_vec_values]) + '\n')
    
meta.close()
vec.close()

You can look at the embedding at"https://projector.tensorflow.org/"